In [1]:
from func_adl_servicex import ServiceXSourceUpROOT

In [2]:
dataset_uproot = "user.kchoi:user.kchoi.ttHML_80fb_ttbar"
ds = ServiceXSourceUpROOT(dataset_uproot, "nominal")
data = ds.Select("lambda e: {'lep_pt_1': e.lep_Pt_1/1000.0}") \
    .AsAwkwardArray() \
    .value()


In [3]:
data

<Array [{lep_pt_1: 29.7}, ... {lep_pt_1: 179}] type='550283 * {"lep_pt_1": float32}'>